In [109]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

np.random.seed(16)

# Elementary SLAE

In [112]:
shape = (100, 4)
X = np.random.uniform(-20, 20, shape)
v = np.array([[1, -2, 4, 1]])
Y = np.dot(X, v.T)

noise = np.random.normal(0, 1, (shape[0], 1))
Y += noise

In [113]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)


In [104]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, Y_train)

LinearRegression()

In [105]:
lin_reg.coef_

array([[ 1.00812853, -1.99018429,  3.97974285,  1.01115793]])

In [108]:
Y_pred = lin_reg.predict(X_test)
print('Mean squared error: %.2f'
      % mean_squared_error(Y_test, Y_pred))

Mean squared error: 0.69


# Custom Regressor

In [114]:
from sklearn.base import BaseEstimator, RegressorMixin

In [ ]:
class MaeLinearRegression(BaseEstimator, RegressorMixin):
    def fit(x, y, lr, delta):
        w = np.zeros((1, x.shape[1]))
        while 
        grad = 2 * X.T * (X.dot(w))
        